<a href="https://www.kaggle.com/code/kunrittyhe/used-car-prices-autogluon-full-train?scriptVersionId=198567082" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install autogluon.tabular[all]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.8/234.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 

# Imports #

In [2]:
import numpy as np 
import pandas as pd 

from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularDataset, TabularPredictor

In [3]:
df_train = pd.read_csv("/kaggle/input/playground-series-s4e9/train.csv", index_col="id")
df_test = pd.read_csv("/kaggle/input/playground-series-s4e9/test.csv", index_col="id")

In [4]:
df_train

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
id,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500
...,...,...,...,...,...,...,...,...,...,...,...,...
188528,Cadillac,Escalade ESV Platinum,2017,49000,Gasoline,420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Beige,None reported,Yes,27500
188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,2018,28600,Gasoline,385.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes,30000
188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,2021,13650,Gasoline,469.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,White,Black,None reported,Yes,86900


# Preprocessing #

In [5]:
def clean(df):
    df["fuel_type"] = df["fuel_type"].replace("Plug-In Hybrid", "Hybrid")
    df["clean_title"] = df["clean_title"].fillna("No") #To treat feature as binary
    
    return df

Extract horsepower, liters, and cylinders from `engine` feature.

Extract automatic, manual, or dct (dual clutch transmission) from `transmission` feature. 

In [6]:
def extract_features(df):
    #Engine features
    df["engine_horsepower"] = df["engine"].str.extract(r'(\d+\.\d+|\d+)\s*HP').astype(float)
    df["engine_liters"] = df["engine"].str.extract(r'(\d+\.\d+|\d+)\s*L').astype(float)
    df_cylinders = df['engine'].str.extract(r'(\d+)\s*Cylinder|V(\d+)', expand=False)
    df['engine_cylinders'] = df_cylinders[0].fillna(df_cylinders[1]) #Combine both regex searches
    
    df = df.drop("engine", axis=1)
    
    #Transmission features
    df["transmission_automatic"] = df["transmission"].str.contains(r'Automatic|A/T|AT', case=False)
    df["transmission_manual"] = df["transmission"].str.contains(r'Manual|M/T|MT', case=False)
    df["transimssion_dct"] = df["transmission"].str.contains('Dual', case=False)
    
    return df

Bundle feature extraction and cleaning into one pipeline

In [7]:
def preprocess(df_train, df_test):
    df = pd.concat([df_train, df_test], axis=0)
    
    df = clean(df)
    df = extract_features(df)
    
    df_train = df.loc[df_train.index]
    df_test = df.loc[df_test.index]
    return df_train, df_test

In [8]:
df_train, df_test = preprocess(df_train, df_test)

In [9]:
df_train.head()

,brand,model,model_year,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,price,engine_horsepower,engine_liters,engine_cylinders,transmission_automatic,transmission_manual,transimssion_dct
id,,,,,,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,A/T,Yellow,Gray,None reported,Yes,4200.0,172.0,1.6,4,True,False,False
1,Lincoln,LS V8,2002,143250,Gasoline,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999.0,252.0,3.9,8,True,False,False
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,A/T,Blue,Gray,None reported,Yes,13900.0,320.0,5.3,8,True,False,False
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000.0,420.0,5.0,8,False,False,True
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,7-Speed A/T,Black,Beige,None reported,Yes,97500.0,208.0,2.0,4,True,False,False


# Modeling #

First, create holdout set for validation. Random seed is to keep consistent OOF predictions throughout different notebooks.

In [10]:
#Convert for autogluon
ds_train = TabularDataset(df_train)
ds_test = TabularDataset(df_test)


In [11]:
xgb_params = {'max_depth': 6, 'min_child_weight': 7, 'colsample_bytree': 0.21577008076093662, 
              'subsample': 0.7812835287449484, 'learning_rate': 0.01459299700503753, 'max_leaves': 44, 
              'n_estimators': 1287, 'reg_alpha': 0.017095752487029176, 'reg_lambda': 98.93396229323028}
cat_params = {'iterations': 1040, 'depth': 8, 'learning_rate': 0.01812522069947833, 
              'l2_leaf_reg': 8.217612632114935, 'bagging_temperature': 0.5059198086110822, 
              'border_count': 225}

In [12]:
params = {
    'NN_TORCH': {},
    'GBM': {}, 
    'CAT': cat_params, 
    'XGB': xgb_params, 
    'FASTAI': {}, 
    'RF': {}, 
    'XT': {}, 
}

In [13]:
#Disabled dynamic stacking since it was done on this dataset in other notebooks
model = TabularPredictor(label="price", eval_metric="rmse").fit(
    ds_train, 
    presets="best_quality", 
    hyperparameters=params, 
    dynamic_stacking=False, 
    num_stack_levels=1, 
    time_limit=3600*11
)

No path specified. Models will be saved in: "AutogluonModels/ag-20240927_200012"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Jun 27 20:43:36 UTC 2024
CPU Count:          4
Memory Avail:       30.11 GB / 31.36 GB (96.0%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ... Time limit = 39600s
AutoGluon will save models to "AutogluonModels/ag-20240927_200012"
Train Data Rows:    188533
Train Data Columns: 16
Label Column:       price
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (2954083.0, 2000.0, 43878.01618, 78819.52225)
	If 'regr

In [14]:
model.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-72494.986432,root_mean_squared_error,111.263230,13477.153925,0.004571,0.645587,3,True,16
1,XGBoost_BAG_L2,-72515.605417,root_mean_squared_error,89.229503,10784.926909,13.394612,2283.612822,2,True,14
2,CatBoost_BAG_L2,-72526.478443,root_mean_squared_error,77.640862,9597.155540,1.805971,1095.841453,2,True,11
3,WeightedEnsemble_L2,-72539.058062,root_mean_squared_error,75.839632,8501.664084,0.004741,0.349997,2,True,8
4,XGBoost_BAG_L1,-72643.965113,root_mean_squared_error,40.675209,4402.918279,40.675209,4402.918279,1,True,6
5,CatBoost_BAG_L1,-72743.252917,root_mean_squared_error,3.371210,1791.497179,3.371210,1791.497179,1,True,3
6,NeuralNetFastAI_BAG_L2,-72837.438886,root_mean_squared_error,84.952692,9465.931190,9.117801,964.617102,2,True,13
7,LightGBM_BAG_L2,-73064.464783,root_mean_squared_error,77.947948,8540.378375,2.113058,39.064287,2,True,9
8,NeuralNetTorch_BAG_L2,-73142.888809,root_mean_squared_error,79.155411,9012.551154,3.320520,511.237067,2,True,15
9,LightGBM_BAG_L1,-73166.187460,root_mean_squared_error,2.484230,35.847877,2.484230,35.847877,1,True,1


In [15]:
predictions = model.predict(ds_test)

In [16]:
predictions

id
188533    17749.984375
188534    76099.296875
188535    55383.699219
188536    27398.126953
188537    29851.152344
              ...     
314218    27368.267578
314219    49239.257812
314220    21520.796875
314221    17148.921875
314222    36965.292969
Name: price, Length: 125690, dtype: float32

In [17]:
predictions.to_csv("submission.csv")